In [1]:
import numpy as np
import pandas as pd 
import tqdm.notebook as tqdm
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords         
from nltk.stem import PorterStemmer  

In [2]:
tqdm.tqdm_notebook.pandas()
pd.set_option('display.max_colwidth', None)

In [3]:
STOP_WORDS = stopwords.words('english') 

In [4]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')

In [5]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [6]:
PUNCTUATIONS = '"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
def clean_text(text):
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    text = re.sub(r'#', '', text)
    return text

def remove_stop_words_and_puntuation(tokenized_text):
    text_clean = []
    
    for word in tokenized_text:
        if (word not in PUNCTUATIONS and word not in STOP_WORDS):
            text_clean.append(word)
            
    return text_clean
    
def stemm_text(tokenized_text):
    text_stemm = []
    
    stemmer = PorterStemmer()
    
    for word in tokenized_text:
        text_stemm.append(stemmer.stem(word))
        
    return text_stemm

def process_text(text):
    text = clean_text(text)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    text = tokenizer.tokenize(text)
    
    text = remove_stop_words_and_puntuation(text)
    text = stemm_text(text)
            
    return text

def create_frequency_map(data: pd.DataFrame):
    data_dict = data.to_dict()

    frequency_map = {}

    for idx in tqdm.tqdm_notebook(range(len(data_dict["id"]))):

        target = data_dict["target"][idx]
        words = data_dict["text"][idx].split(" ")
        for word in words:
            target = data_dict["target"][idx]
            
            if (word.lower(), target) in frequency_map:
                frequency_map[(word.lower(), target)] += 1
            else:
                frequency_map[(word.lower(), target)] = 1

    return frequency_map
    

In [8]:
def sigmoid(z): 
    
    h =  1/(1 + np.exp(-z))
    
    return h


def gradientDescent(x, y, theta, alpha, num_iters):
    
    m = len(x)
    J = 0

    for i in tqdm.tqdm_notebook(range(0, num_iters)):

        z = np.dot(x,theta)
        h = sigmoid(z)
        J = (-1 / m * (np.dot(y.T,np.log(h)) + np.dot((1-y).T, np.log(1-h))))
        theta = theta - (alpha/m) * np.dot(x.T , (h-y))

    J = float(J[0])
    return J, theta


def extract_features(text, freqs):

    word_l = process_text(text)

    x = np.zeros((1, 3)) 

    x[0,0] = 1 

    for word in word_l:
        
        if (word, 1.0) in freqs:
            x[0,1] += freqs[(word, 1.0)]
        
        if (word, 0.0) in freqs:
            x[0,2] += freqs[(word, 0.0)]

    return x


In [9]:
def predict_text(text, freqs, theta):

    x = extract_features(text, freqs)

    y_pred = sigmoid(np.dot(x,theta))
    
    if y_pred[0] > 0.5:
        return 1
    else:
        return 0

In [10]:
FREQUENCY_MAP = create_frequency_map(train_data)

  0%|          | 0/7613 [00:00<?, ?it/s]

In [11]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [12]:
X = np.zeros((len(train_data), 3))

for idx in range(len(X)):
    X[idx, :]= extract_features(train_data['text'][idx], FREQUENCY_MAP)

In [13]:
y = [[n] for n in train_data['target'].to_numpy()]
y = np.array(y)

In [14]:
J, theta = gradientDescent(X, y, np.zeros((3, 1)), 1e-9, 100000)
J

  0%|          | 0/100000 [00:00<?, ?it/s]

0.6613537437794654

In [15]:
submission = {'id': [], 'target':[]}
for n in tqdm.tqdm_notebook(range(len(test_data))):
    row = test_data.loc[n, :]

    submission['id'].append(row['id'])
    submission['target'].append(predict_text(row['text'], FREQUENCY_MAP, theta))


  0%|          | 0/3263 [00:00<?, ?it/s]

In [16]:
df_submission = pd.DataFrame(submission)
df_submission

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [17]:
df_submission.to_csv('submission.csv', index=False)